## Can I Run?

In this example we will use a quantum computer to solve a classical Machine Learning problem: Given a set of constraints, if there are too many of them, then what is the right combination such that Malai is able to go for a run.

For this example, we will take a look at 3 constraints:
- Is it raining outside? - If yes, then I can't run
- Is it cool outside? - If yes, then I can run
- Need to drop son in school? - If yes, then I can't run
- Time - I am an avid runner. I will run even if it is raining as long as it is between 5am and 7am

Now to Define the different constraints:
- If it rains (1), then malai will run only if the time is between 5am and 7am (1)
- If it does not rain (0), and the time is not between 5am and 7am (0), he will run only if he does not have to drop his son (0) and if it is cool (1)
- Finally, if it does not rain (0), but the time is between 5am and 7am (1), he will run.

In [1]:
def cir(rain, time, drop, temp):
    if rain:
        return time
    elif not(rain):
        if time:
            return time
        elif not(time) and drop:
            return time
        elif not(time) and not(drop) and temp:
            return temp
        else:
            return time

In [2]:
from IPython.display import Image
#Image(filename='img.png')

First we start by importing all the DWave Packages and initialising the DWave solver.
We add the constraints that we discussed previously.

In [3]:
import dwavebinarycsp

csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)

#Our constraints in this case are rain, temperature, drop and time.
csp.add_constraint(cir, ['rain', 'time', 'drop', 'temp'])

In [4]:
!dwave solvers

Solver: DW_2000Q_2_1
  Parameters:
    anneal_offsets: A list of anneal offsets for each working qubit (NaN if u...
    anneal_schedule: A piecewise linear annealing schedule specified by a list...
    annealing_time: A positive integer that sets the duration (in microsecond...
    answer_mode: ?
    auto_scale: ?
    beta: Boltzmann distribution parameter. Only used when postproc...
    chains: Defines which qubits represent the same logical variable ...
    flux_biases: A list of flux biases for each working qubit.
    flux_drift_compensation: A boolean for whether to activate the flux drift compensa...
    h_gain_schedule: A piecewise linear h-gain schedule specified by a list of...
    initial_state: The initial states to use for the reverse anneal feature.
    max_answers: ?
    num_reads: ?
    num_spin_reversal_transforms: ?
    postprocess: either 'sampling' or 'optimization'
    programming_thermalization: An integer that gives the time (in microseconds) to wait ...
    readou

In [5]:
!dwave ping

Using endpoint: https://cloud.dwavesys.com/sapi
Using solver: DW_2000Q_5
Submitted problem ID: 921a46bc-4bec-4400-8c01-2719a597534e

Wall clock time:
 * Solver definition fetch: 2829.745 ms
 * Problem submit and results fetch: 1445.095 ms
 * Total: 4274.840 ms

QPU timing:
 * anneal_time_per_run = 20 us
 * total_real_time = 10015 us
 * qpu_access_overhead_time = 1764 us
 * qpu_readout_time_per_sample = 274 us
 * qpu_anneal_time_per_sample = 20 us
 * readout_time_per_run = 274 us
 * qpu_access_time = 10015 us
 * run_time_chip = 315 us
 * post_processing_overhead_time = 333 us
 * total_post_processing_time = 333 us
 * qpu_programming_time = 9700 us
 * qpu_delay_time_per_sample = 21 us
 * qpu_sampling_time = 315 us


In [11]:
%matplotlib inline

In [12]:
bqm = dwavebinarycsp.stitch(csp)

print('The BQM is')
bqm.quadratic

The BQM is


Running it on the DWave System

In [13]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())

import time
start_time=time.time()
response = sampler.sample(bqm, num_reads=5000)
request_time=time.time()-start_time

print("Total Time Taken to get result: {} sec".format(round(request_time, 2)))

Total Time Taken to get result: 5.12 sec


Getting the Answers

In [14]:
# total = 0
# for sample, energy, occurrences in response.data(['sample', 'energy', 'num_occurrences']):
#     total = total + occurrences
#     if energy == min_energy:
#         time = 'business hours' if sample['time'] else 'evenings'
#         location = 'office' if sample['location'] else 'home'
#         length = 'short' if sample['length'] else 'long'
#         mandatory = 'mandatory' if sample['mandatory'] else 'optional'
#         print("{}: During {} at {}, you can schedule a {} meeting that is {}".format(occurrences, time, location, length, mandatory))
#     print("Total occurrences: ", total)

valid, invalid, data = 0, 0, []
for datum in response.data(['sample', 'energy', 'num_occurrences']):
    if (csp.check(datum.sample)):
        valid = valid+datum.num_occurrences
        for i in range(datum.num_occurrences):
            data.append((datum.sample, datum.energy, '1'))
    else:
        invalid = invalid+datum.num_occurrences
        for i in range(datum.num_occurrences):
            data.append((datum.sample, datum.energy, '0'))
print("\nNo. of Low Energy state answers: {}".format(valid))
print("No. of High Energy State Answers: {}".format(invalid))


No. of Low Energy state answers: 5000
No. of High Energy State Answers: 0


In [16]:
results_dict={}
total_time=0
qpu_time=0
num=0

for sample, energy in response.data(['sample', 'energy']):
    #import ipdb; ipdb.set_trace()
    rain_value=sample['rain']
    temp_value=sample['temp']
    drop_value=sample['drop']
    time_value=sample['time']
    #import ipdb; ipdb.set_trace()
    if (rain_value, time_value, drop_value, temp_value) not in results_dict:
        results_dict[(rain_value, time_value, drop_value, temp_value)] = energy
    
    total_time=total_time+response.info['timing']['total_real_time']
    qpu_time=qpu_time+response.info['timing']['qpu_anneal_time_per_sample']
    num=num+1
#import ipdb; ipdb.set_trace()
for result in results_dict:
    if result[0]:
        if result[1]:
            print("Even though it is raining, since the time is fine, I can run. \n {} Energy: {}".format(result,results_dict[result]))
        else:
            print("It's raining, but the time is not right, so I won't run. \n {} Energy: {}".format(result,results_dict[result]))
    elif not(result[0]):
        if result[1]:
            print("It is not raining and the time is perfect, I will go for a run. \n {} Energy: {}".format(result,results_dict[result]))
        elif not(result[1]) and result[2]:
            print("Even though it is not raining, the time is not right and I have to drop my son off at school. I wil not run. \n {} Energy: {}".format(result,results_dict[result]))
        elif not(result[1]) and not(result[2]) and result[3]:
            print("Neither is it raining, nor is it hot and I don't have to drop my son off at school. I will go for a run. \n {} Energy: {}".format(result,results_dict[result]))
        elif not(result[1]) and not(result[2]) and not(result[3]):
            print("It isn't raining, but it is hot so I won't go for a run. \n {} Energy: {}".format(result,results_dict[result]))
    else:
        print('Incorrect Result: \n {} Energy: {results_dict[result]}'.format(result,results_dict[result]))

print("\nTotal Real Time Required by Quantum Computer: {} seconds".format(total_time/1000000))
#print(f"\nTime per Iteration: {(total_time/1000000)} milli seconds")
print("Average Time Required in Quantum Computer per Iteration: {} microseconds!".format(qpu_time/num))

Even though it is raining, since the time is fine, I can run. 
 (1, 1, 0, 1) Energy: 0.0
It is not raining and the time is perfect, I will go for a run. 
 (0, 1, 1, 0) Energy: 0.0
Even though it is raining, since the time is fine, I can run. 
 (1, 1, 1, 0) Energy: 0.0
It is not raining and the time is perfect, I will go for a run. 
 (0, 1, 0, 1) Energy: 0.0
Even though it is raining, since the time is fine, I can run. 
 (1, 1, 0, 0) Energy: 0.0
Neither is it raining, nor is it hot and I don't have to drop my son off at school. I will go for a run. 
 (0, 0, 0, 1) Energy: -1.9999999967268423e-07
It is not raining and the time is perfect, I will go for a run. 
 (0, 1, 1, 1) Energy: 0.0
It is not raining and the time is perfect, I will go for a run. 
 (0, 1, 0, 0) Energy: 0.0
Even though it is raining, since the time is fine, I can run. 
 (1, 1, 1, 1) Energy: 0.0

Total Real Time Required by Quantum Computer: 17.429324 seconds
Average Time Required in Quantum Computer per Iteration: 20.0 m